## Basic RL usage

- ..
- our basic environment configurations are specified in the env_config yaml file.

In [2]:
from importlib import reload
import yaml
import logging

logging.basicConfig(level=logging.INFO)

### Initializing environments

Initializing an environment can be done with the `BaseEnv` class. This class uses the Nocturne simulator under the hood to create a basic RL interface based on a given traffic scenario (e.g. `example_scenario.json`).

Note:

---
> ⚠️ The `env_config.yaml` file defines the action space, observation space, traffic scenarios we want to use, and more.
---


In [ ]:
from nocturne.envs.base_env import BaseEnv

# Load environment settings
with open(f"../configs/env_config.yaml", "r") as stream:
    env_config = yaml.safe_load(stream)

# Make environment
env = BaseEnv(
    config=env_config,
)

### Interacting with the environment

The classic agent-environment loop of reinforcement learning is implemented as follows:

In [ ]:
# Reset
obs_dict = env.reset()
agent_ids = [agent_id for agent_id in obs_dict.keys()]
dead_agent_ids = []
num_agents = len(agent_ids)

for step in range(100):

    # Sample actions
    action_dict = {
        agent_id: env.action_space.sample() 
        for agent_id in enumerate(agent_ids)
        if agent_id not in dead_agent_ids
    }

    # Step in env
    obs_dict, rew_dict, done_dict, info_dict = env.step(action_dict)

    # Update dead agents
    for agent_id, is_done in done_dict.items():
        if is_done and agent_id not in dead_agent_ids:
            dead_agent_ids.append(agent_id)

    # Reset if all agents are done
    if done_dict["__all__"]:
        obs_dict = env.reset()
        dead_agent_ids = []

env.close()


---
> ⚠️ The default env returns dictionaries 
---

### Accessing information about the environment

In [ ]:
env.observation_space


In [ ]:
env.action_space


### 
